In [9]:
import pandas as pd


In [10]:
sales_df = pd.read_csv("/Users/sovanpradhan/Desktop/HCLHackathon/prod_store_sales_line_items (1).csv")
products_df = pd.read_csv("products.csv")
promotions_df = pd.read_csv("promotion_details.csv")


In [14]:
sales_df["promotion_id"] = sales_df["promotion_id"].fillna("0")


In [16]:
sales_df

,line_item_id,transaction_id,product_id,quantity,promotion_id,line_item_amount,line_item_amount_raw
0,1,T000001,P0004,4,0,9106.76,"₹9,106.76"
1,2,T000002,P0011,1,0,3753.01,"INR 3,753.01"
2,3,T000002,P0011,3,0,11259.03,"11,259.03"
3,4,T000002,P0012,2,0,221.38,INR 221.38
4,5,T000002,P0002,2,0,1048.42,1048.42
...,...,...,...,...,...,...,...
294,366,T000119,P0003,4,0,1455.24,"INR 1,455.24"
295,367,T000119,P0006,3,PR002,6097.56,"INR 6,097.56"
296,368,T000119,P0020,3,0,1513.35,"1,513.35"
297,369,T000119,P0023,2,0,827.18,827.18


In [17]:
products_df.head()

,product_id,product_name,product_category,unit_price,current_stock_level,current_inventory_count
0,P0001,Product P0001,Home,472.26,264,245
1,P0002,Product P0002,Electronics,487.58,348,338
2,P0003,Product P0003,Apparel,352.24,222,203
3,P0004,Product P0004,Electronics,315.05,411,420
4,P0005,Product P0005,Electronics,351.00,32,12


In [18]:
sales_df = sales_df.merge(
    products_df[["product_id", "product_category"]],
    on="product_id",
    how="left"
)


In [19]:
promotions_df.head()

,promotion_id,promotion_name,start_date,end_date,discount_percentage,applicable_category
0,PR001,Weekend Electronics Flash Sale,2024-10-05,2024-10-06,0.15,Electronics
1,PR002,Apparel Season Kickoff,2024-10-10,2024-10-20,0.20,Apparel
2,PR003,Grocery Essentials Deal,2024-10-01,2024-10-07,0.10,Grocery
3,PR004,Home & Kitchen Savings,2024-11-01,2024-11-15,0.18,Home
4,PR005,Beauty Bundle Discount,2024-11-18,2024-11-30,0.12,Beauty


In [20]:
sales_df = sales_df.merge(
    promotions_df,
    on="promotion_id",
    how="left"
)


In [21]:
promoted_sales = sales_df[sales_df["promotion_id"].notna()]
baseline_sales = sales_df[sales_df["promotion_id"].isna()]


In [22]:

promo_metrics = promoted_sales.groupby(
    ["promotion_id", "promotion_name", "product_category"]
).agg(
    promoted_units=("quantity", "sum"),
    promoted_revenue=("line_item_amount", "sum")
).reset_index()


In [23]:
promo_metrics = promoted_sales.groupby(
    ["promotion_id", "promotion_name", "product_category"]
).agg(
    promoted_units=("quantity", "sum"),
    promoted_revenue=("line_item_amount", "sum")
).reset_index()


In [24]:
baseline_metrics = baseline_sales.groupby(
    "product_category"
).agg(
    baseline_units=("quantity", "sum"),
    baseline_revenue=("line_item_amount", "sum")
).reset_index()


In [25]:
promo_metrics = promo_metrics.merge(
    baseline_metrics,
    on="product_category",
    how="left"
)


In [26]:
promo_metrics["sales_lift_percentage"] = (
    (promo_metrics["promoted_units"] - promo_metrics["baseline_units"]) /
    promo_metrics["baseline_units"]
) * 100


In [27]:
top_3_promotions = promo_metrics.sort_values(
    by="sales_lift_percentage",
    ascending=False
).head(3)


In [28]:
print("🔥 TOP 3 MOST EFFECTIVE PROMOTIONS 🔥")
print(top_3_promotions)


🔥 TOP 3 MOST EFFECTIVE PROMOTIONS 🔥
  promotion_id                  promotion_name product_category  \
0        PR001  Weekend Electronics Flash Sale          Apparel   
1        PR001  Weekend Electronics Flash Sale           Beauty   
2        PR001  Weekend Electronics Flash Sale      Electronics   

   promoted_units  promoted_revenue  baseline_units  baseline_revenue  \
0               7           8485.60             NaN               NaN   
1               1            388.85             NaN               NaN   
2               3           7345.20             NaN               NaN   

   sales_lift_percentage  
0                    NaN  
1                    NaN  
2                    NaN  
